### Google Analytics 4 and BigQuery with Pandas
#### [GA4 Public Dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce&t=events_20210131&page=table)


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Use a service account for authentication onto GCP
credentials = service_account.Credentials.from_service_account_file('path_to_secrets.json')

# The project id to which the service account has access
project_id = 'your-bigquery-project-id'

# Establish a connection to the client
client = bigquery.Client(credentials= credentials, project=project_id)

# Perform a query
query = client.query(
"""
SELECT ...
""")

#Convert output into a pandas dataframe
df = query.to_dataframe()
df